In [1]:
#Imports
import os
import pathlib
import json
import PIL
import cv2
import numpy as np
import tensorflow as tf 
from PIL import Image
from google.protobuf import text_format
from object_detection.utils import config_util
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_utils
from object_detection.protos import pipeline_pb2
from object_detection.builders import model_builder
import IPython.display as display

In [4]:
#Format annotations to use supercategories instead of categories
f = open('./data/TACO-master/data/annotations_orig.json')
json_file = json.load(f)
annotations = json_file['annotations']
categories = json_file['categories']
for annotation in annotations:
    if annotation['category_id'] == 0:
        annotation['category_id'] = 1
    elif annotation['category_id'] == 1:
        annotation['category_id'] = 2
    elif annotation['category_id'] == 2:
        annotation['category_id'] = 3
    elif annotation['category_id'] == 3:
        annotation['category_id'] = 3
    elif annotation['category_id'] == 4:
        annotation['category_id'] = 4
    elif annotation['category_id'] == 5:
        annotation['category_id'] = 4
    elif annotation['category_id'] == 6:
        annotation['category_id'] = 4
    elif annotation['category_id'] == 7:
        annotation['category_id'] = 5
    elif annotation['category_id'] == 8:
        annotation['category_id'] = 5
    elif annotation['category_id'] == 9:
        annotation['category_id'] = 6
    elif annotation['category_id'] == 10:
        annotation['category_id'] = 7
    elif annotation['category_id'] == 11:
        annotation['category_id'] = 7
    elif annotation['category_id'] == 12:
        annotation['category_id'] = 7
    elif annotation['category_id'] == 13:
        annotation['category_id'] = 8
    elif annotation['category_id'] == 14:
        annotation['category_id'] = 8
    elif annotation['category_id'] == 15:
        annotation['category_id'] = 8
    elif annotation['category_id'] == 16:
        annotation['category_id'] = 8
    elif annotation['category_id'] == 17:
        annotation['category_id'] = 8
    elif annotation['category_id'] == 18:
        annotation['category_id'] = 8
    elif annotation['category_id'] == 19:
        annotation['category_id'] = 8
    elif annotation['category_id'] == 20:
        annotation['category_id'] = 9
    elif annotation['category_id'] == 21:
        annotation['category_id'] = 9
    elif annotation['category_id'] == 22:
        annotation['category_id'] = 9
    elif annotation['category_id'] == 23:
        annotation['category_id'] = 9
    elif annotation['category_id'] == 24:
        annotation['category_id'] = 9
    elif annotation['category_id'] == 25:
        annotation['category_id'] = 10
    elif annotation['category_id'] == 26:
        annotation['category_id'] = 11
    elif annotation['category_id'] == 27:
        annotation['category_id'] = 12
    elif annotation['category_id'] == 28:
        annotation['category_id'] = 12
    elif annotation['category_id'] == 29:
        annotation['category_id'] = 13
    elif annotation['category_id'] == 30:
        annotation['category_id'] = 14
    elif annotation['category_id'] == 31:
        annotation['category_id'] = 14
    elif annotation['category_id'] == 32:
        annotation['category_id'] = 14
    elif annotation['category_id'] == 33:
        annotation['category_id'] = 14
    elif annotation['category_id'] == 34:
        annotation['category_id'] = 15
    elif annotation['category_id'] == 35:
        annotation['category_id'] = 15
    elif annotation['category_id'] == 36:
        annotation['category_id'] = 16
    elif annotation['category_id'] == 37:
        annotation['category_id'] = 16
    elif annotation['category_id'] == 38:
        annotation['category_id'] = 16
    elif annotation['category_id'] == 39:
        annotation['category_id'] = 16
    elif annotation['category_id'] == 40:
        annotation['category_id'] = 16
    elif annotation['category_id'] == 41:
        annotation['category_id'] = 16
    elif annotation['category_id'] == 42:
        annotation['category_id'] = 16
    elif annotation['category_id'] == 43:
        annotation['category_id'] = 17
    elif annotation['category_id'] == 44:
        annotation['category_id'] = 17
    elif annotation['category_id'] == 45:
        annotation['category_id'] = 17
    elif annotation['category_id'] == 46:
        annotation['category_id'] = 17
    elif annotation['category_id'] == 47:
        annotation['category_id'] = 17
    elif annotation['category_id'] == 48:
        annotation['category_id'] = 18
    elif annotation['category_id'] == 49:
        annotation['category_id'] = 19
    elif annotation['category_id'] == 50:
        annotation['category_id'] = 20
    elif annotation['category_id'] == 51:
        annotation['category_id'] = 21
    elif annotation['category_id'] == 52:
        annotation['category_id'] = 22
    elif annotation['category_id'] == 53:
        annotation['category_id'] = 23
    elif annotation['category_id'] == 54:
        annotation['category_id'] = 24
    elif annotation['category_id'] == 55:
        annotation['category_id'] = 25
    elif annotation['category_id'] == 56:
        annotation['category_id'] = 25
    elif annotation['category_id'] == 57:
        annotation['category_id'] = 26
    elif annotation['category_id'] == 58:
        annotation['category_id'] = 27
    elif annotation['category_id'] == 59:
        annotation['category_id'] = 28

f.close()
with open("./data/TACO-master/data/annotations_new.json", 'w') as f:
    json.dump(json_file, f)

In [5]:
#Splitting annotiations into training and testing sets and generating tfrecords files
tf_models_dir = "./TensorFlow/models"
splitting_script = "./data/TACO-master/detector/split_dataset.py"
tfrecord_creation_script = "./TensorFlow/models/research/object_detection/dataset_tools/create_coco_tf_record.py"
tf_record_dir = "./tf_records/"
dataset_path = "./data/TACO-master/data/"
annotations_training = dataset_path + "annotations_0_train.json"
annotations_testing = dataset_path + "annotations_0_test.json"
annotations_val = dataset_path + "annotations_0_val.json"

%run $splitting_script "--dataset_dir" $dataset_path

#Generate tfrecords
%run $tfrecord_creation_script "--logtostderr" "--train_image_dir" $dataset_path "--val_image_dir" $dataset_path "--test_image_dir" $dataset_path "--train_annotations_file" $annotations_training "--val_annotations_file" $annotations_val "--testdev_annotations_file" $annotations_testing "--output_dir" $tf_record_dir

I0704 00:13:09.419782 19976 create_coco_tf_record.py:399] Found groundtruth annotations. Building annotations index.
I0704 00:13:09.421785 19976 create_coco_tf_record.py:411] 0 images are missing annotations.
I0704 00:13:09.422784 19976 create_coco_tf_record.py:441] On image 0 of 1350
I0704 00:13:10.757084 19976 create_coco_tf_record.py:441] On image 100 of 1350
I0704 00:13:12.362817 19976 create_coco_tf_record.py:441] On image 200 of 1350
I0704 00:13:13.893161 19976 create_coco_tf_record.py:441] On image 300 of 1350
I0704 00:13:15.597761 19976 create_coco_tf_record.py:441] On image 400 of 1350
I0704 00:13:17.074095 19976 create_coco_tf_record.py:441] On image 500 of 1350
I0704 00:13:18.598437 19976 create_coco_tf_record.py:441] On image 600 of 1350
I0704 00:13:20.217802 19976 create_coco_tf_record.py:441] On image 700 of 1350
I0704 00:13:22.068218 19976 create_coco_tf_record.py:441] On image 800 of 1350
I0704 00:13:23.821613 19976 create_coco_tf_record.py:441] On image 900 of 1350
I07

In [2]:
#Print command to be used for training the model
my_model_dir = "C:/Users/Randon/Desktop/Projects/ai-trash-detector/models/my_models/640x640ssd"
config_file = my_model_dir + "/pipeline.config"
training_script = "C:/Users/Randon/Desktop/Projects/ai-trash-detector/TensorFlow/models/research/object_detection/model_main_tf2.py"
print("python {} --model_dir={} --num_train_steps=250000 --sample_1_of_n_eval_examples=1  --pipeline_config_path={} --alsologtostderr".format(training_script, my_model_dir, config_file))


python C:/Users/Randon/Desktop/Projects/ai-trash-detector/TensorFlow/models/research/object_detection/model_main_tf2.py --model_dir=C:/Users/Randon/Desktop/Projects/ai-trash-detector/models/my_models/640x640ssd --num_train_steps=250000 --sample_1_of_n_eval_examples=1  --pipeline_config_path=C:/Users/Randon/Desktop/Projects/ai-trash-detector/models/my_models/640x640ssd/pipeline.config --alsologtostderr
